In [ ]:
!wget https://data.mendeley.com/public-files/datasets/tywbtsjrjv/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/file_downloaded

--2022-09-20 20:32:05--  https://data.mendeley.com/public-files/datasets/tywbtsjrjv/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/file_downloaded
Resolving data.mendeley.com (data.mendeley.com)... 162.159.130.86, 162.159.133.86
Connecting to data.mendeley.com (data.mendeley.com)|162.159.130.86|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://prod-dcd-datasets-public-files-eu-west-1.s3.eu-west-1.amazonaws.com/d29ed9b2-8a5d-4663-8a82-c9174f2c7066 [following]
--2022-09-20 20:32:06--  https://prod-dcd-datasets-public-files-eu-west-1.s3.eu-west-1.amazonaws.com/d29ed9b2-8a5d-4663-8a82-c9174f2c7066
Resolving prod-dcd-datasets-public-files-eu-west-1.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-public-files-eu-west-1.s3.eu-west-1.amazonaws.com)... 52.218.37.32
Connecting to prod-dcd-datasets-public-files-eu-west-1.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-public-files-eu-west-1.s3.eu-west-1.amazonaws.com)|52.218.37.32|:443... connected.
HTTP request sent,

In [ ]:
!unzip /content/file_downloaded

In [1]:
import os
from platform import python_version
import warnings
import time
import datetime as dt
from sklearn.metrics import classification_report, confusion_matrix
import multiprocessing as mp
import shutil

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

import pandas as pd
import numpy as np
import seaborn as sn

from PIL import Image
import xml.etree.ElementTree as ET
import psutil
import random

warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
BASE_DIR = "/content/Plant_leave_diseases_dataset_without_augmentation"


In [ ]:
# !rm -rf "/content/test/Background_without_leaves"

In [ ]:
# !rm -rf "/content/Plant_leave_diseases_dataset_without_augmentation"
# !rm -rf "/content/test"

In [ ]:
src = "/content/Plant_leave_diseases_dataset_without_augmentation"


In [ ]:
# sorted(os.listdir(src))

In [ ]:
selected = ['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Cherry___Powdery_mildew',
 'Cherry___healthy',
 'Corn___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn___Common_rust',
 'Corn___Northern_Leaf_Blight',
 'Corn___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato___Target_Spot',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Tomato_mosaic_virus',
 'Tomato___healthy'
]

In [ ]:
len(selected)

34

In [ ]:
from tqdm import tqdm

In [ ]:
def split_dataset(name):
  f = name
  src = "/content/Plant_leave_diseases_dataset_without_augmentation"
  images = os.listdir(src + '/' + f)
  test_images = np.random.choice(images,size=int(len(images) * 0.3),replace=False)
  train_images = [x for x in images if x not in test_images]
  test_dir = "/content/data/test"
  train_dir = "/content/data/train"
  os.makedirs(test_dir+'/'+ f)
  os.makedirs(train_dir+'/'+ f)
  for test_image in test_images:
    test_image_dir_src = src + '/' + f + '/' + test_image
    test_image_dir_mv = test_dir + '/' + f 
    if(os.path.exists(test_image_dir_src)):
      # print(f"moving from {test_image_dir_src} to {test_image_dir_mv}")
      shutil.move(test_image_dir_src,test_image_dir_mv)
  for train_image in train_images:
    train_image_dir_src = src + '/' + f + '/' + train_image
    train_image_dir_mv = train_dir + '/' + f 
    if(os.path.exists(train_image_dir_src)):
      # print(f"moving from {train_image_dir_src} to {train_image_dir_mv}")
      shutil.move(train_image_dir_src,train_image_dir_mv)


In [ ]:
for plantname in tqdm(selected):
  split_dataset(plantname)

100%|██████████| 34/34 [00:02<00:00, 12.45it/s]


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
IMG_SHAPE = (224,224)
BATCH_SIZE = 128
test_dir = "/content/data/test"
train_dir = "/content/data/train"

In [ ]:
train_datagen = ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        brightness_range=[0.9,1.1],
        horizontal_flip=True,
        rescale=(1/255.)
)
test_datagen = ImageDataGenerator(rescale=1/255.)

In [ ]:
train_data = train_datagen.flow_from_directory(train_dir,target_size=IMG_SHAPE,batch_size=BATCH_SIZE,class_mode="categorical")
test_data = test_datagen.flow_from_directory(test_dir,target_size=IMG_SHAPE,batch_size=BATCH_SIZE,class_mode="categorical")

Found 31869 images belonging to 34 classes.
Found 13638 images belonging to 34 classes.


In [ ]:
checkpoint = ModelCheckpoint('plant_disease_checkpoint.h5',
                             monitor='val_accuracy',
                             save_best_only=True,
                             verbose=1,
                             mode='auto',
                             save_weights_only=False,
                             period=1)

earlystop  = EarlyStopping(monitor="val_accuracy",min_delta=0.0001,patience=20,verbose=1,mode='auto',baseline=None,restore_best_weights=True)
reducelr = ReduceLROnPlateau(monitor='val_accuracy',
                             factor=np.sqrt(.1),
                             patience=5,
                             verbose=1,
                             mode='auto',
                             min_delta=.0001,
                             cooldown=0,
                             min_lr=0.0000001)

In [ ]:
classes = selected
learning_rate = 0.0001
seed = random.randint(1, 1000)

In [ ]:
mobile_net_base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(IMG_SHAPE[0],IMG_SHAPE[1], 3))
x = mobile_net_base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(1280, activation='relu',  kernel_initializer=glorot_uniform(seed), bias_initializer='zeros')(x)
x = BatchNormalization()(x)
outputs = Dense(len(classes), activation='softmax', kernel_initializer='random_uniform', bias_initializer='zeros')(x)
model = tf.keras.Model(inputs = mobile_net_base_model.input,outputs=outputs)
optimizer = Adam(lr=learning_rate)
loss = "categorical_crossentropy"

In [ ]:
for layer in model.layers:
    layer.trainable = True

model.compile(optimizer=optimizer,
              loss=loss,
              metrics=["accuracy"])

In [ ]:
epochs = 10
history = model.fit(
    train_data,
    steps_per_epoch=len(train_data),
    validation_data=(test_data),
    validation_steps=len(test_data),
    epochs=epochs,
    callbacks=[reducelr, earlystop,checkpoint])

Epoch 1/10
498/498 [==============================] - ETA: 0s - loss: 0.3278 - accuracy: 0.9066
Epoch 1: val_accuracy improved from -inf to 0.61307, saving model to plant_disease_checkpoint.h5
498/498 [==============================] - 487s 969ms/step - loss: 0.3278 - accuracy: 0.9066 - val_loss: 1.3411 - val_accuracy: 0.6131 - lr: 1.0000e-04
Epoch 2/10
498/498 [==============================] - ETA: 0s - loss: 0.0586 - accuracy: 0.9813
Epoch 2: val_accuracy improved from 0.61307 to 0.77475, saving model to plant_disease_checkpoint.h5
498/498 [==============================] - 485s 973ms/step - loss: 0.0586 - accuracy: 0.9813 - val_loss: 0.7809 - val_accuracy: 0.7747 - lr: 1.0000e-04
Epoch 3/10
498/498 [==============================] - ETA: 0s - loss: 0.0345 - accuracy: 0.9896
Epoch 3: val_accuracy improved from 0.77475 to 0.87234, saving model to plant_disease_checkpoint.h5
498/498 [==============================] - 482s 967ms/step - loss: 0.0345 - accuracy: 0.9896 - val_loss: 0.4543

In [2]:
project_folder_dir = "/content/drive/MyDrive/Projects/leaf_detection"

In [3]:

!pip install tensorflowjs 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 8.9 MB/s 
     |████████████████████████████████| 180 kB 69.7 MB/s 
     |████████████████████████████████| 40 kB 6.8 MB/s 
     |████████████████████████████████| 217 kB 65.4 MB/s 
     |████████████████████████████████| 145 kB 70.9 MB/s 
     |████████████████████████████████| 51 kB 7.5 MB/s 
     |████████████████████████████████| 85 kB 5.2 MB/s 
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3


In [4]:
!mkdir model
!tensorflowjs_converter --input_format keras /content/drive/MyDrive/Projects/leaf_detection/leaf_model.h5 model/

In [5]:
!zip -r model.zip model 

  adding: model/ (stored 0%)
  adding: model/group1-shard2of4.bin (deflated 7%)
  adding: model/group1-shard3of4.bin (deflated 7%)
  adding: model/group1-shard1of4.bin (deflated 7%)
  adding: model/group1-shard4of4.bin (deflated 7%)
  adding: model/model.json (deflated 96%)


In [ ]:
model.save(project_folder_dir + '/model')

In [ ]:
model.save(project_folder_dir + '/leaf_model.h5')

In [ ]:
abr1 = "/content/image (55).JPG"
abr2 = "/content/image (77).JPG"
peb1 = "/content/image (27).JPG"
peb2 = "/content/image (5).JPG"

In [ ]:
import cv2

In [ ]:
def predict(model,dir):
  image = cv2.imread(dir)
  image = cv2.resize(image,IMG_SHAPE)
  image = np.array(image)
  image = image / 255.
  preds = model.predict(np.array([image]))
  return np.argmax(preds,axis=-1)[0]

In [ ]:
classes[predict(model,abr1)]

'Apple___Black_rot'

In [ ]:
classes[predict(model,abr2)]

'Apple___Black_rot'

In [ ]:
classes[predict(model,peb1)]

'Potato___Early_blight'

In [ ]:
classes[predict(model,peb2)]

'Pepper,_bell___healthy'